In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from scipy.ndimage import gaussian_filter

csv_path      = "path_to_input_csv"
output_folder = "path_to_output_folder"
image_size    = 256
sigma         = 5

os.makedirs(output_folder, exist_ok=True)

df = pd.read_csv(csv_path)
cluster_indices = range(6)  

labels = set()
for i in cluster_indices:
    labels |= set(df[f"Cluster_{i}_Type"].dropna().unique())
labels.discard("")  

accumulators = {
    label: np.zeros((image_size, image_size), dtype=np.float32)
    for label in labels
}

for _, row in df.iterrows():
    for i in cluster_indices:
        label = row.get(f"Cluster_{i}_Type")
        if not isinstance(label, str) or label == "":
            continue

        cx_norm = row.get(f"Cluster_{i}_Centroid_X")
        cy_norm = row.get(f"Cluster_{i}_Centroid_Y")
        if pd.isna(cx_norm) or pd.isna(cy_norm):
            continue

        cx = int((cx_norm + 5) / 10 * image_size)
        cy = int((cy_norm + 5) / 10 * image_size)
        if 0 <= cx < image_size and 0 <= cy < image_size:
            accumulators[label][cy, cx] += 1.0

for label, heatmap_acc in accumulators.items():

    heatmap_blur = gaussian_filter(heatmap_acc, sigma=sigma)
    if heatmap_blur.max() > 0:

        heatmap_img = (heatmap_blur / heatmap_blur.max() * 255).astype(np.uint8)
    else:
        heatmap_img = np.zeros_like(heatmap_blur, dtype=np.uint8)

    out_path = os.path.join(output_folder, f"heatmap_{label}.png")
    cv2.imwrite(out_path, heatmap_img)

print(f"Created {len(labels)} aggregate heatmaps in '{output_folder}'.")


In [ ]:
import pandas as pd

csv_path = "path_to_input_csv"
df = pd.read_csv(csv_path)

cluster_type_cols = [f"Cluster_{i}_Type" for i in range(6)]
unique_labels = set()
for col in cluster_type_cols:
    unique_labels.update(df[col].dropna().unique())

img_count = {lab: 0 for lab in unique_labels}   
occ_count = {lab: 0 for lab in unique_labels}  

for _, row in df.iterrows():
    present = set(row[col] for col in cluster_type_cols if pd.notna(row[col]))
    for lab in present:
        img_count[lab] += 1
    for col in cluster_type_cols:
        lab = row[col]
        if pd.notna(lab):
            occ_count[lab] += 1

print(f"\nLabel statistics for {df['Scene_ID'].nunique()} scenes:")
print(f"{'Label':25s} {'Scenes':>8s} {'Occurrences':>12s}")
print("-------------------------------------------------------------------")
for lab in sorted(unique_labels, key=img_count.get, reverse=True):
    print(f"{lab:25s} {img_count[lab]:8d} {occ_count[lab]:12d}")

print("\nTOTAL OCCURRENCES ACROSS ALL LABELS:", sum(occ_count.values()))


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from scipy.ndimage import gaussian_filter

base_path = "path_to_furniture_data"
csv_path = "path_to_input_csv"
output_folder = "path_to_output_folder"
image_size = 256
sigma = 10

df = pd.read_csv(csv_path)
cluster_indices = range(6)  

labels = set()
for i in cluster_indices:
    labels.update(df[f"Cluster_{i}_Type"].dropna().unique())
for label in labels:
    os.makedirs(os.path.join(output_folder, f"heatmap_{label}"), exist_ok=True)

for _, row in df.iterrows():
    scene_id = row["Scene_ID"]

    for i in cluster_indices:
        label = row.get(f"Cluster_{i}_Type")
        if pd.isna(label) or label == "":
            continue

        cx_norm = row.get(f"Cluster_{i}_Centroid_X", None)
        cy_norm = row.get(f"Cluster_{i}_Centroid_Y", None)
        if pd.isna(cx_norm) or pd.isna(cy_norm):
            continue

        cx = int((cx_norm + 5) / 10 * image_size) 
        cy = int((cy_norm + 5) / 10 * image_size)

        if not (0 <= cx < image_size and 0 <= cy < image_size):
            continue

        heatmap = np.zeros((image_size, image_size), dtype=np.float32)
        heatmap[cy, cx] = 1.0
        heatmap = gaussian_filter(heatmap, sigma=sigma)
        heatmap = (heatmap / heatmap.max() * 255).astype(np.uint8)

        save_path = os.path.join(output_folder, f"heatmap_{label}", f"{scene_id}.png")
        cv2.imwrite(save_path, heatmap)

print("Heatmaps created from cluster data.")